In [1]:
!git clone https://github.com/pthengtr/kcw-analytics.git

Cloning into 'kcw-analytics'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (64/64), done.
Receiving objects: 100% (109/109), 78.64 KiB | 1.31 MiB/s, done.
remote: Total 109 (delta 47), reused 68 (delta 18), pack-reused 0 (from 0)
Resolving deltas: 100% (47/47), done.


In [2]:
!cd /content/kcw-analytics && git pull origin main

From https://github.com/pthengtr/kcw-analytics
 * branch            main       -> FETCH_HEAD
Already up to date.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import pandas as pd

folder = "/content/drive/MyDrive/kcw_analytics/01_raw"

data = {}

for file in os.listdir(folder):
    if file.endswith(".csv"):
        path = os.path.join(folder, file)
        data[file] = pd.read_csv(
            path,
            dtype={
              "BCODE": "string",
              "ITEMNO": "string",
              "BILLNO": "string",
            },
            encoding="utf-8-sig",
            low_memory=False   # stops chunk guessing
        )
        print(f"Loaded: {file} -> {data[file].shape}")

Loaded: raw_hq_pimas_purchase_bills.csv -> (82716, 49)
Loaded: raw_hq_simas_sales_bills.csv -> (481937, 49)
Loaded: raw_hq_pidet_purchase_lines.csv -> (246580, 41)
Loaded: raw_hq_sidet_sales_lines.csv -> (1187205, 38)
Loaded: raw_hq_icmas_products.csv -> (114755, 94)
Loaded: raw_syp_pimas_purchase_bills.csv -> (2721, 49)
Loaded: raw_syp_simas_sales_bills.csv -> (10368, 49)
Loaded: raw_syp_sidet_sales_lines.csv -> (30374, 38)
Loaded: raw_syp_pidet_purchase_lines.csv -> (25555, 41)
Loaded: raw_inventory_hq_2024.csv -> (4983, 8)


In [25]:
import sys
import importlib

# ensure repo is on path
repo_path = "/content/kcw-analytics"
if repo_path not in sys.path:
    sys.path.append(repo_path)

# import the module (NOT individual functions)
import src.kcw.utils as utils

# reload to pick up latest .py changes
importlib.reload(utils)

get_vat_sales_lines_last_purchase_nonvat = utils.get_vat_sales_lines_last_purchase_nonvat
audit_bcode_vat_sales_last_purchase = utils.audit_bcode_vat_sales_last_purchase

In [26]:
YEAR = 2026

vat_sales_last_purchase_nonvat_hq = get_vat_sales_lines_last_purchase_nonvat(
    data, year=YEAR, source="hq"
)

vat_sales_last_purchase_nonvat_syp = get_vat_sales_lines_last_purchase_nonvat(
    data, year=YEAR, source="syp"
)


In [27]:
vat_sales_last_purchase_nonvat_hq

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
23,1226927,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-011,10,1,12050690,...,NaN,7.0,590.0,NaN,7LPNT,N,CRRF,N,N,N
24,1226858,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-004,10,1,12050729,...,NaN,7.0,350.0,NaN,7SPNT,N,BM,N,N,N
25,1227016,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-013,10,1,12051272,...,NaN,7.0,205.0,NaN,7SPNT,N,KCY,N,N,N
27,1226983,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-012,20,2,12051561,...,NaN,7.0,140.0,NaN,7ICE,N,BM,N,N,N
28,1226984,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-012,30,3,12051563,...,NaN,7.0,90.0,NaN,7ICE,N,BM,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,1240464,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-098,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5096,1240465,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-099,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5097,1240467,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-100,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5099,1240501,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TAD6901-535,20,2,30051639,...,NaN,7.0,490.0,NaN,7LAZ1,N,CRRK,N,N,N


In [28]:
vat_sales_last_purchase_nonvat_hq_tar_only = vat_sales_last_purchase_nonvat_hq[
    vat_sales_last_purchase_nonvat_hq["BILLNO"].str.match(r"^TAR", na=False)
]
vat_sales_last_purchase_nonvat_hq_tar_only

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
1191,1230247,0,SJ,2026-01-07 00:00:00,1.0,2026-01-07,TAR6901-104,10,1,13026861,...,NaN,7.0,40.0,NaN,7000,N,NaN,N,N,N
1409,1231020,0,SJ,2026-01-08 00:00:00,1.0,2026-01-08,TAR6901-134,10,1,12053734,...,NaN,7.0,7920.0,NaN,7000,N,NaN,N,N,N
2268,1233212,0,SJ,2026-01-12 00:00:00,1.0,2026-01-12,TAR6901-191,40,4,19040111,...,NaN,7.0,20.0,NaN,7000,N,7ANP,N,N,N
2970,1235322,0,SJ,2026-01-16 00:00:00,1.0,2026-01-16,TAR6901-255,50,5,12051662,...,NaN,7.0,150.0,NaN,7000,N,NaN,N,N,N
3650,1237036,0,SJ,2026-01-19 00:00:00,1.0,2026-01-19,TAR6901-309,10,1,12053734,...,NaN,7.0,5940.0,NaN,7000,N,NaN,N,N,N


In [29]:
vat_sales_last_purchase_nonvat_syp

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
453,1693718,1,SJ,2026-01-08 00:00:00,1,2026-01-08,3TR6901-005,30,3,13036340,...,NaN,7.0,9.0,NaN,7ชจร,N,LK,N,N,N
648,1744098,0,SJ,2026-01-12 00:00:00,1,2026-01-12,3TAR6901-072,70,7,13027010,...,NaN,7.0,6.8,NaN,7000,N,LK,N,N,N
713,1754135,1,SJ,2026-01-13 00:00:00,1,2026-01-13,3TR6901-008,20,2,09011683,...,NaN,7.0,430.0,NaN,7อลย,N,SRI,N,N,N
878,1774163,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,20,2,05018690,...,NaN,7.0,60.0,NaN,7ชจร,N,SSWK,N,N,N
888,1774167,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,60,6,10050185,...,NaN,7.0,108.0,NaN,7ชจร,N,YOKO,N,N,N
889,1774162,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,10,1,12050484,...,NaN,7.0,240.0,NaN,7ชจร,N,SJP,N,N,N
1048,1774591,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,60,6,14010442,...,NaN,7.0,60.0,NaN,7ชจร,N,RJS,N,N,N
1053,1774588,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,40,4,17016070,...,NaN,7.0,160.0,NaN,7ชจร,N,LK,N,N,N
1089,1774587,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,30,3,33010369,...,NaN,7.0,200.0,NaN,7ชจร,N,MC,N,N,N
1123,1784711,1,SJ,2026-01-19 00:00:00,1,2026-01-19,3TR6901-013,60,6,13051791,...,NaN,7.0,28.0,NaN,7ชจร,N,SMA,N,N,N


In [30]:
vat_sales_last_purchase_nonvat_syp_tar_only = vat_sales_last_purchase_nonvat_syp[
    vat_sales_last_purchase_nonvat_syp["BILLNO"].str.match(r"^3TAR", na=False)
]
vat_sales_last_purchase_nonvat_syp_tar_only

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
648,1744098,0,SJ,2026-01-12 00:00:00,1,2026-01-12,3TAR6901-072,70,7,13027010,...,NaN,7.0,6.8,NaN,7000,N,LK,N,N,N
1334,1805220,0,SJ,2026-01-22 00:00:00,1,2026-01-22,3TAR6901-149,40,4,12052373,...,NaN,7.0,620.0,NaN,7000,N,NaN,N,N,N
1416,1815285,0,SJ,2026-01-23 00:00:00,1,2026-01-23,3TAR6901-154,10,1,12052692,...,NaN,7.0,750.0,NaN,7000,N,7CNH,N,N,N
1468,1815286,0,SJ,2026-01-23 00:00:00,1,2026-01-23,3TAR6901-154,20,2,32050343,...,NaN,7.0,1650.0,NaN,7000,N,KT,N,N,N
1480,1825359,0,SJ,2026-01-24 00:00:00,1,2026-01-24,3TAR6901-165,20,2,05057622,...,NaN,7.0,320.0,NaN,7000,N,NaN,N,N,NaN
1487,1825320,0,SJ,2026-01-24 00:00:00,1,2026-01-24,3TAR6901-161,30,3,13026790,...,NaN,7.0,40.0,NaN,7000,N,YSR,N,N,N
1531,1825352,0,SJ,2026-01-24 00:00:00,1,2026-01-24,3TAR6901-164,50,5,32050343,...,NaN,7.0,1650.0,NaN,7000,N,NaN,N,N,N


In [44]:
vat_sales_last_purchase_nonvat_hq_cleaned = vat_sales_last_purchase_nonvat_hq[
    ~vat_sales_last_purchase_nonvat_hq["BILLNO"]
        .astype(str)
        .str.startswith(("TAR", "CN"), na=False)
]

vat_sales_last_purchase_nonvat_hq_cleaned

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
23,1226927,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-011,10,1,12050690,...,NaN,7.0,590.0,NaN,7LPNT,N,CRRF,N,N,N
24,1226858,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-004,10,1,12050729,...,NaN,7.0,350.0,NaN,7SPNT,N,BM,N,N,N
25,1227016,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-013,10,1,12051272,...,NaN,7.0,205.0,NaN,7SPNT,N,KCY,N,N,N
27,1226983,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-012,20,2,12051561,...,NaN,7.0,140.0,NaN,7ICE,N,BM,N,N,N
28,1226984,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-012,30,3,12051563,...,NaN,7.0,90.0,NaN,7ICE,N,BM,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,1240464,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-098,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5096,1240465,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-099,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5097,1240467,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-100,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5099,1240501,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TAD6901-535,20,2,30051639,...,NaN,7.0,490.0,NaN,7LAZ1,N,CRRK,N,N,N


In [43]:
vat_sales_last_purchase_nonvat_syp_cleaned = vat_sales_last_purchase_nonvat_syp[
    ~vat_sales_last_purchase_nonvat_syp["BILLNO"]
        .astype(str)
        .str.startswith(("3TAR","3CN"), na=False)
]

vat_sales_last_purchase_nonvat_syp_cleaned

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
453,1693718,1,SJ,2026-01-08 00:00:00,1,2026-01-08,3TR6901-005,30,3,13036340,...,NaN,7.0,9.0,NaN,7ชจร,N,LK,N,N,N
713,1754135,1,SJ,2026-01-13 00:00:00,1,2026-01-13,3TR6901-008,20,2,09011683,...,NaN,7.0,430.0,NaN,7อลย,N,SRI,N,N,N
878,1774163,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,20,2,05018690,...,NaN,7.0,60.0,NaN,7ชจร,N,SSWK,N,N,N
888,1774167,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,60,6,10050185,...,NaN,7.0,108.0,NaN,7ชจร,N,YOKO,N,N,N
889,1774162,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,10,1,12050484,...,NaN,7.0,240.0,NaN,7ชจร,N,SJP,N,N,N
1048,1774591,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,60,6,14010442,...,NaN,7.0,60.0,NaN,7ชจร,N,RJS,N,N,N
1053,1774588,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,40,4,17016070,...,NaN,7.0,160.0,NaN,7ชจร,N,LK,N,N,N
1089,1774587,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,30,3,33010369,...,NaN,7.0,200.0,NaN,7ชจร,N,MC,N,N,N
1123,1784711,1,SJ,2026-01-19 00:00:00,1,2026-01-19,3TR6901-013,60,6,13051791,...,NaN,7.0,28.0,NaN,7ชจร,N,SMA,N,N,N
1414,1815252,1,SJ,2026-01-23 00:00:00,1,2026-01-23,3TR6901-015,20,2,12051665,...,NaN,7.0,30.0,NaN,7ชจร,N,SMA,N,N,N


In [33]:
#sales_one, pidet_one = audit_bcode_vat_sales_last_purchase(data, "12050690", YEAR, "hq")

In [46]:
vat_sales_last_purchase_nonvat_hq_cleaned[['BCODE', 'BILLNO', 'QTY', 'MTP', 'UI', 'AMOUNT', 'ACCT_NO', 'CANCELED','ISVAT', 'LAST_PURCHASE_ISVAT']]

,BCODE,BILLNO,QTY,MTP,UI,AMOUNT,ACCT_NO,CANCELED,ISVAT,LAST_PURCHASE_ISVAT
23,12050690,TAD6901-011,1.0,1.0,หน่วย,590.0,CRRF,N,Y,N
24,12050729,TAD6901-004,1.0,1.0,หน่วย,350.0,BM,N,Y,N
25,12051272,TAD6901-013,1.0,1.0,ชุด,205.0,KCY,N,Y,N
27,12051561,TAD6901-012,2.0,1.0,หน่วย,140.0,BM,N,Y,N
28,12051563,TAD6901-012,2.0,1.0,หน่วย,90.0,BM,N,Y,N
...,...,...,...,...,...,...,...,...,...,...
5095,30050835,TD6901-098,1.0,1.0,หน่วย,90.0,CRRK,N,Y,N
5096,30050835,TD6901-099,1.0,1.0,หน่วย,90.0,CRRK,N,Y,N
5097,30050835,TD6901-100,1.0,1.0,หน่วย,90.0,CRRK,N,Y,N
5099,30051639,TAD6901-535,2.0,1.0,เส้น,490.0,CRRK,N,Y,N


In [47]:
vat_sales_last_purchase_nonvat_syp_cleaned

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
453,1693718,1,SJ,2026-01-08 00:00:00,1,2026-01-08,3TR6901-005,30,3,13036340,...,NaN,7.0,9.0,NaN,7ชจร,N,LK,N,N,N
713,1754135,1,SJ,2026-01-13 00:00:00,1,2026-01-13,3TR6901-008,20,2,09011683,...,NaN,7.0,430.0,NaN,7อลย,N,SRI,N,N,N
878,1774163,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,20,2,05018690,...,NaN,7.0,60.0,NaN,7ชจร,N,SSWK,N,N,N
888,1774167,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,60,6,10050185,...,NaN,7.0,108.0,NaN,7ชจร,N,YOKO,N,N,N
889,1774162,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,10,1,12050484,...,NaN,7.0,240.0,NaN,7ชจร,N,SJP,N,N,N
1048,1774591,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,60,6,14010442,...,NaN,7.0,60.0,NaN,7ชจร,N,RJS,N,N,N
1053,1774588,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,40,4,17016070,...,NaN,7.0,160.0,NaN,7ชจร,N,LK,N,N,N
1089,1774587,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,30,3,33010369,...,NaN,7.0,200.0,NaN,7ชจร,N,MC,N,N,N
1123,1784711,1,SJ,2026-01-19 00:00:00,1,2026-01-19,3TR6901-013,60,6,13051791,...,NaN,7.0,28.0,NaN,7ชจร,N,SMA,N,N,N
1414,1815252,1,SJ,2026-01-23 00:00:00,1,2026-01-23,3TR6901-015,20,2,12051665,...,NaN,7.0,30.0,NaN,7ชจร,N,SMA,N,N,N


In [77]:
import pandas as pd
import numpy as np

def enrich_sales_with_avcost_and_newbillno(
    data: dict,
    sales_lines: pd.DataFrame,
    *,
    source: str,  # <-- NEW: "hq" or "syp"
    purchase_key: str = "raw_hq_pidet_purchase_lines.csv",
    # common columns
    bcode_col: str = "BCODE",
    date_col: str = "BILLDATE",
    # sales qty columns (kept for signature compatibility)
    sales_qty_col: str = "QTY",
    sales_mtp_col: str = "MTP",
    # purchase columns for cost calc
    purchase_amount_col: str = "AMOUNT",
    purchase_qty_col: str = "QTY",
    purchase_mtp_col: str = "MTP",
    # bill cols (kept for signature compatibility)
    billno_col: str = "BILLNO",
    # output
    output_cols: tuple = (
        "BCODE", "BILLDATE", "BILLNO", "DETAIL", "QTY", "MTP", "UI", "AMOUNT", "ACCT_NO",
        "CANCELED", "ISVAT", "LAST_PURCHASE_ISVAT", "AV_COST"
    ),
) -> pd.DataFrame:
    """
    Takes already-filtered sales lines and:
      1) Looks up weighted-average UNIT cost as-of sale date from purchases in data[purchase_key].
      2) Adds NEW_BILLNO:
         - HQ  -> RV[BDyy][mm]-[seq]
         - SYP -> 3RV[BDyy][mm]-[seq]
         - Sequence resets per MONTH
         - Within a month, sequence increments by 1 for each new DATE,
           and for each additional chunk of 10 lines on the same DATE.
      3) Returns only requested columns (plus NEW_BILLNO).
    """

    # --- validate source ---
    source = source.lower().strip()
    if source not in ("hq", "syp"):
        raise ValueError("source must be 'hq' or 'syp'")

    bill_prefix = "RV" if source == "hq" else "3RV"

    # --- helpers ---
    def _clean_cols(df: pd.DataFrame) -> pd.DataFrame:
        out = df.copy()
        out.columns = (
            out.columns.astype(str)
            .str.replace("\ufeff", "", regex=False)
            .str.strip()
        )
        return out

    def _clean_bcode(s: pd.Series) -> pd.Series:
        return s.astype(str).str.strip()

    # --- copy & clean ---
    sales = _clean_cols(sales_lines).copy()
    purch = _clean_cols(data[purchase_key]).copy()

    # Ensure key columns exist
    for col in [bcode_col, date_col]:
        if col not in sales.columns:
            raise KeyError(f"sales_lines missing required column: {col}")
        if col not in purch.columns:
            raise KeyError(f"purchase data missing required column: {col}")

    # Clean BCODE + dates
    sales[bcode_col] = _clean_bcode(sales[bcode_col])
    purch[bcode_col] = _clean_bcode(purch[bcode_col])

    sales[date_col] = pd.to_datetime(sales[date_col], errors="coerce")
    purch[date_col] = pd.to_datetime(purch[date_col], errors="coerce")

    sales = sales.dropna(subset=[bcode_col, date_col]).copy()
    purch = purch.dropna(subset=[bcode_col, date_col]).copy()

    sales = sales[sales[bcode_col] != ""].copy()
    purch = purch[purch[bcode_col] != ""].copy()

    # Reduce purchases to only needed BCODEs (perf)
    purch = purch[purch[bcode_col].isin(sales[bcode_col].unique())].copy()

    # --- 1) AV_COST as-of sale date (LAST 10 purchase lines) ---
    for c in [purchase_amount_col, purchase_qty_col, purchase_mtp_col]:
        purch[c] = pd.to_numeric(purch[c], errors="coerce")

    purch = purch.dropna(subset=[purchase_amount_col, purchase_qty_col, purchase_mtp_col]).copy()
    purch = purch[(purch[purchase_qty_col] != 0) & (purch[purchase_mtp_col] > 0)].copy()

    # cost basis per purchase line: AMOUNT / QTY * MTP  (your confirmed formula)
    purch["__COST_BASIS__"] = (
        purch[purchase_amount_col] / purch[purchase_qty_col]
    ) * purch[purchase_mtp_col]

    purch = purch.sort_values([bcode_col, date_col], kind="mergesort").reset_index(drop=True)

    # Rolling sums over LAST 10 purchase rows per BCODE
    g = purch.groupby(bcode_col, sort=False)
    purch["__ROLL_COST__"] = g["__COST_BASIS__"].transform(lambda s: s.rolling(10, min_periods=1).sum())
    purch["__ROLL_MTP__"]  = g[purchase_mtp_col].transform(lambda s: s.rolling(10, min_periods=1).sum())

    purch["__AVCOST__"] = purch["__ROLL_COST__"] / purch["__ROLL_MTP__"]

    purch_key = purch[[bcode_col, date_col, "__AVCOST__"]].copy()

    sales_sorted = sales.sort_values([date_col, bcode_col], kind="mergesort").reset_index(drop=True)
    purch_key = purch_key.sort_values([date_col, bcode_col], kind="mergesort").reset_index(drop=True)

    merged = pd.merge_asof(
        sales_sorted,
        purch_key,
        left_on=date_col,
        right_on=date_col,
        by=bcode_col,
        direction="backward",
        allow_exact_matches=True,
    )

    merged["AV_COST"] = merged["__AVCOST__"]
    merged.drop(columns="__AVCOST__", inplace=True)

    # --- 2) NEW_BILLNO (monthly reset, daily + 10-line chunk increment) ---
    merged["__DATEKEY__"] = merged[date_col].dt.normalize()
    merged["__MONTHKEY__"] = merged["__DATEKEY__"].dt.to_period("M")

    merged = merged.sort_values(["__DATEKEY__", bcode_col], kind="mergesort").reset_index(drop=True)

    day_chunk = merged.groupby("__DATEKEY__", sort=False).cumcount() // 20

    day_tbl = (
        pd.DataFrame({
            "__MONTHKEY__": merged["__MONTHKEY__"],
            "__DATEKEY__": merged["__DATEKEY__"],
            "day_chunk": day_chunk
        })
        .groupby(["__MONTHKEY__", "__DATEKEY__"], sort=False)["day_chunk"]
        .max()
        .rename("MAX_CHUNK")
        .reset_index()
        .sort_values(["__MONTHKEY__", "__DATEKEY__"], kind="mergesort")
    )

    day_tbl["DAY_START_SEQ0"] = (
        day_tbl.groupby("__MONTHKEY__")["MAX_CHUNK"]
        .transform(lambda x: (x + 1).cumsum() - (x + 1))
    )

    start_map = day_tbl.set_index(["__MONTHKEY__", "__DATEKEY__"])["DAY_START_SEQ0"]
    merged["__DAY_START_SEQ0__"] = start_map.loc[
        list(zip(merged["__MONTHKEY__"], merged["__DATEKEY__"]))
    ].to_numpy()

    seq = (merged["__DAY_START_SEQ0__"] + day_chunk + 1).astype(int)

    bd_yy = ((merged["__DATEKEY__"].dt.year + 543) % 100).astype(int)
    mm = merged["__DATEKEY__"].dt.month.astype(int)

    merged["NEW_BILLNO"] = (
        bill_prefix
        + bd_yy.map(lambda x: f"{x:02d}")
        + mm.map(lambda x: f"{x:02d}")
        + "-"
        + seq.map(lambda x: f"{x:03d}")
    )

    merged.drop(columns=["__DATEKEY__", "__MONTHKEY__", "__DAY_START_SEQ0__"], inplace=True)

    # --- 3) Output ---
    for col in output_cols:
        if col not in merged.columns:
            merged[col] = pd.NA

    return merged[list(output_cols) + ["NEW_BILLNO"]].copy()


In [78]:
out_syp = enrich_sales_with_avcost_and_newbillno(
    data=data,
    sales_lines=vat_sales_last_purchase_nonvat_syp_cleaned,
    source = "syp"
)

out_hq = enrich_sales_with_avcost_and_newbillno(
    data=data,
    sales_lines=vat_sales_last_purchase_nonvat_hq_cleaned,
    source = "hq"
)


In [79]:
out_syp

,BCODE,BILLDATE,BILLNO,DETAIL,QTY,MTP,UI,AMOUNT,ACCT_NO,CANCELED,ISVAT,LAST_PURCHASE_ISVAT,AV_COST,NEW_BILLNO
0,13036340,2026-01-08,3TR6901-005,"แหวนสปริง 3/8"" ดำ",6.0,1.0,ตัว,9.0,LK,N,Y,N,0.175388,3RV6901-001
1,09011683,2026-01-13,3TR6901-008,ไส้กรองเครื่อง เหล็ก ก.16NF ไฮเทค3H HO7D-FM1J ...,1.0,1.0,หน่วย,430.0,SRI,N,Y,N,350.000000,3RV6901-002
2,05018690,2026-01-15,3TR6901-010,หัวแป๊บเบรค D/S620 หัวสิงห์,3.0,1.0,ตัว,60.0,SSWK,N,Y,N,10.850000,3RV6901-003
3,10050185,2026-01-15,3TR6901-010,ยางกันฝุ่นเบรค หน้า-หลัง SUMO YOKO,6.0,1.0,ตัว,108.0,YOKO,N,Y,N,9.100000,3RV6901-003
4,12050484,2026-01-15,3TR6901-010,"สกรูใบมีดดันหน้า F 5/8"" x 2"" สีทอง",12.0,1.0,หน่วย,240.0,SJP,N,Y,N,10.200000,3RV6901-003
5,14010442,2026-01-17,3TR6901-012,ลวดเชื่อมทองเหลือง เส้นเล็ก 2.6 mm.,2.0,1.0,เส้น,60.0,RJS,N,Y,N,2318.911359,3RV6901-004
6,17016070,2026-01-17,3TR6901-012,แกนสกรูมิลดำ ไม่หัว M16-70m ก.1.50 ธรรมดา,8.0,1.0,ตัว,160.0,LK,N,Y,N,10.011600,3RV6901-004
7,33010369,2026-01-17,3TR6901-012,"หัวสาย+ปลอก IMS 0808 GORS 90"" MK",1.0,1.0,ชุด,200.0,MC,N,Y,N,101.944444,3RV6901-004
8,13051791,2026-01-19,3TR6901-013,หัวแป็บเบรค ตัวผู้ 1/4 หยาบ หัวสิงห์,1.0,1.0,หัว,28.0,SMA,N,Y,N,14.000000,3RV6901-005
9,12051665,2026-01-23,3TR6901-015,ตาไก่ทองเหลือง 1/4 SAK,2.0,1.0,หน่วย,30.0,SMA,N,Y,N,5.900000,3RV6901-006


In [80]:
out_hq

,BCODE,BILLDATE,BILLNO,DETAIL,QTY,MTP,UI,AMOUNT,ACCT_NO,CANCELED,ISVAT,LAST_PURCHASE_ISVAT,AV_COST,NEW_BILLNO
0,12050690,2026-01-01,TAD6901-011,ปั้มAC/ปั้มแย็กทั้งลูก F/6610 แท้สั่ง,1.0,1.0,หน่วย,590.0,CRRF,N,Y,N,350.000000,RV6901-001
1,12050729,2026-01-01,TAD6901-004,ไส้กรองปั้มขาว F/6610 ตัวเล็ก New arrival,1.0,1.0,หน่วย,350.0,BM,N,Y,N,200.000000,RV6901-001
2,12051272,2026-01-01,TAD6901-013,เม็ดลูกปืน+หลอดยานัตถ์ พมล. F/6600 New Holland,1.0,1.0,ชุด,205.0,KCY,N,Y,N,95.625000,RV6901-001
3,12051561,2026-01-01,TAD6901-012,สกรู 6 หุน ยาว 3 นิ้ว FORD เกลียวละเอ BM,2.0,1.0,หน่วย,140.0,BM,N,Y,N,42.857129,RV6901-001
4,12051563,2026-01-01,TAD6901-012,บู๊ชใบมีดตัดหญ้า CMT รู 6 หุน BM,2.0,1.0,หน่วย,90.0,BM,N,Y,N,19.484541,RV6901-001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,30050835,2026-01-26,TD6901-098,"กรองโซล่า เหล็ก L3608-L4508DI,K หัวสิงห์",1.0,1.0,หน่วย,90.0,CRRK,N,Y,N,62.000000,RV6901-032
416,30050835,2026-01-26,TD6901-099,"กรองโซล่า เหล็ก L3608-L4508DI,K หัวสิงห์",1.0,1.0,หน่วย,90.0,CRRK,N,Y,N,62.000000,RV6901-032
417,30050835,2026-01-26,TD6901-100,"กรองโซล่า เหล็ก L3608-L4508DI,K หัวสิงห์",1.0,1.0,หน่วย,90.0,CRRK,N,Y,N,62.000000,RV6901-032
418,30051639,2026-01-26,TAD6901-535,ยางฝาวาว KBT M9540-M108S หัวสิงห์,2.0,1.0,เส้น,490.0,CRRK,N,Y,N,67.500000,RV6901-032


In [84]:
pidet = data["raw_hq_pidet_purchase_lines.csv"].copy()
pidet_filtered = pidet[pidet["BCODE"] == "21050290"].copy()

pidet_filtered["AV_COST"] = (
    pd.to_numeric(pidet_filtered["AMOUNT"], errors="coerce")
    / pd.to_numeric(pidet_filtered["QTY"], errors="coerce")
    * pd.to_numeric(pidet_filtered["MTP"], errors="coerce")
)

pidet_filtered[["BCODE", 'BILLDATE', "QTY", "MTP", "PRICE", "AMOUNT", "AV_COST"]]



,BCODE,BILLDATE,QTY,MTP,PRICE,AMOUNT,AV_COST
1220,21050290,2016-06-29 00:00:00,25.0,1.0,2080.0000,52000.00,2080.000000
2545,21050290,2016-08-27 00:00:00,20.0,1.0,2080.0000,41600.00,2080.000000
22474,21050290,2018-02-08 00:00:00,336.0,1.0,6.2500,2100.00,6.250000
23401,21050290,2018-03-03 00:00:00,10.0,12.0,75.0000,750.00,900.000000
25286,21050290,2018-04-12 00:00:00,12.0,12.0,75.0000,900.00,900.000000
...,...,...,...,...,...,...,...
230410,21050290,2025-08-20 00:00:00,960.0,1.0,5.6666,5439.94,5.666604
231077,21050290,2025-08-21 00:00:00,840.0,1.0,5.6666,4759.94,5.666595
237387,21050290,2025-10-27 00:00:00,1200.0,1.0,5.6666,6799.92,5.666600
242816,21050290,2025-12-15 00:00:00,600.0,1.0,5.6666,3399.96,5.666600


In [56]:
!pip install reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.3 MB/s eta 0:00:00


In [107]:
import os
import pandas as pd

from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from datetime import datetime

TH_MONTHS_ABBR = [
    "ม.ค.", "ก.พ.", "มี.ค.", "เม.ย.", "พ.ค.", "มิ.ย.",
    "ก.ค.", "ส.ค.", "ก.ย.", "ต.ค.", "พ.ย.", "ธ.ค."
]

def thai_date(d) -> str:
    """Return Thai date like: 1 ม.ค. 2569"""
    dt = pd.to_datetime(d).to_pydatetime()
    day = dt.day
    month = TH_MONTHS_ABBR[dt.month - 1]
    year_be = dt.year + 543
    return f"{day} {month} {year_be}"


def _thai_read_0_to_9(n: int) -> str:
    return ["ศูนย์", "หนึ่ง", "สอง", "สาม", "สี่", "ห้า", "หก", "เจ็ด", "แปด", "เก้า"][n]

def _thai_read_integer(num: int) -> str:
    """
    Read integer Thai (Baht style) up to millions+ by looping 'ล้าน'.
    """
    if num == 0:
        return "ศูนย์"

    units = ["", "สิบ", "ร้อย", "พัน", "หมื่น", "แสน"]
    digits = ["ศูนย์", "หนึ่ง", "สอง", "สาม", "สี่", "ห้า", "หก", "เจ็ด", "แปด", "เก้า"]

    def read_under_million(n: int) -> str:
        s = ""
        d = list(map(int, str(n)))
        L = len(d)
        for i, x in enumerate(d):
            pos = L - i - 1  # 0..5
            if x == 0:
                continue

            if pos == 0:  # ones
                if x == 1 and L > 1:
                    s += "เอ็ด"
                else:
                    s += digits[x]
            elif pos == 1:  # tens
                if x == 1:
                    s += "สิบ"
                elif x == 2:
                    s += "ยี่สิบ"
                else:
                    s += digits[x] + "สิบ"
            else:
                s += digits[x] + units[pos]
        return s

    parts = []
    while num > 0:
        parts.append(num % 1_000_000)
        num //= 1_000_000

    # parts[0]=under million, parts[1]=millions group, ...
    out = ""
    for i in range(len(parts)-1, -1, -1):
        n = parts[i]
        if n == 0:
            continue
        chunk = read_under_million(n)
        out += chunk
        if i != 0:
            out += "ล้าน"
    return out


def thai_baht_text(amount) -> str:
    """
    Convert number to Thai Baht text:
    11584.00 -> "หนึ่งหมื่นหนึ่งพันห้าร้อยแปดสิบสี่บาทถ้วน"
    12.50 -> "สิบสองบาทห้าสิบสตางค์"
    """
    x = float(amount) if amount is not None else 0.0
    if x < 0:
        return "ลบ" + thai_baht_text(-x)

    baht = int(x)
    satang = int(round((x - baht) * 100))

    # handle rounding carry
    if satang == 100:
        baht += 1
        satang = 0

    baht_words = _thai_read_integer(baht) + "บาท"
    if satang == 0:
        return baht_words + "ถ้วน"
    else:
        satang_words = _thai_read_integer(satang) + "สตางค์"
        return baht_words + satang_words


def _register_thai_font(font_dir: str | None = None):
    """
    Optional Thai font setup.
    Place these files in font_dir (or current dir):
      - Prompt-Medium.ttf
      - Prompt-Bold.ttf
    """
    if font_dir is None:
        font_dir = "."

    regular = os.path.join(font_dir, "Prompt-Medium.ttf")
    bold = os.path.join(font_dir, "Prompt-Bold.ttf")

    if os.path.exists(regular) and os.path.exists(bold):
        pdfmetrics.registerFont(TTFont("Prompt-Medium", regular))
        pdfmetrics.registerFont(TTFont("Prompt-Bold", bold))
        return ("Prompt-Medium", "Prompt-Bold")

    # fallback (no Thai)
    return ("Helvetica", "Helvetica-Bold")


def _money(x):
    try:
        return f"{float(x):,.2f}"
    except Exception:
        return ""


def build_one_receipt_pdf(group_df: pd.DataFrame, pdf_path: str, *, font_dir: str | None = None):
    # ---- font + styles
    font_regular, font_bold = _register_thai_font(font_dir)
    styles = getSampleStyleSheet()

    title_style = ParagraphStyle(
        "title_th",
        parent=styles["Title"],
        fontName=font_bold,
        fontSize=16,
        leading=18,
        spaceAfter=6
    )
    normal_style = ParagraphStyle(
        "normal_th",
        parent=styles["Normal"],
        fontName=font_regular,
        fontSize=12,
        leading=14
    )
    small_style = ParagraphStyle(
        "small_th",
        parent=styles["Normal"],
        fontName=font_regular,
        fontSize=11,
        leading=13
    )

    from reportlab.lib.enums import TA_RIGHT

    right_style = ParagraphStyle(
      "right_style",
      parent=styles["Normal"],
      fontName=font_regular,
      fontSize=12,
      alignment=TA_RIGHT,   # 👈 RIGHT ALIGN
    )

    right_small_style = ParagraphStyle(
      "right_style",
      parent=styles["Normal"],
      fontName=font_regular,
      alignment=TA_RIGHT,   # 👈 RIGHT ALIGN
    )

    # ---- infer header fields (customize as you like)
    new_billno = str(group_df["NEW_BILLNO"].iloc[0])
    billdate = thai_date(group_df["BILLDATE"].iloc[0])

    # Example: original BILLNO in your cleaned table looks like "3TR6901-010"
    # You can display it too if you want:
    src_billno = str(group_df["BILLNO"].iloc[0])

    branch_text = "สำนักงานใหญ่"
    billno_check = new_billno  # or src_billno, choose the one you mean
    if str(billno_check).startswith("3"):
        branch_text = "สี่แยกพัฒนา"

    # ---- create doc
    doc = SimpleDocTemplate(
        pdf_path,
        pagesize=A4,
        leftMargin=24,
        rightMargin=24,
        topMargin=18,
        bottomMargin=18
    )

    story = []

    # ---- header block (layout similar-ish to your scan)
    story.append(Paragraph("ใบสำคัญรับเงิน", title_style))
    story.append(Paragraph(f"เลขที่: {new_billno}", right_style))
    story.append(Paragraph(f"วันที่: {billdate}", right_style))
    story.append(Spacer(1, 10))

    # Optional buyer fields (blank placeholders)
    story.append(Paragraph('<font name="Prompt-Bold">ข้าพเจ้า:</font> นางสาวนฤมล วิทยผโลทัย (ผู้ขายสินค้า)', normal_style))
    story.append(Paragraph('<font name="Prompt-Bold">ที่อยู่:</font> 305 หมู่ 1 ตำบล ชุมแสง อำเภอ วังจันทร์ จังหวัด ระยอง', normal_style))
    story.append(Paragraph('<font name="Prompt-Bold">เลขประจำตัวผู้เสียภาษี:</font> 1-2001-99001-42-8', normal_style))
    story.append(Spacer(1, 10))
    story.append(Paragraph(f"ได้รับเงินจาก บริษัทเกียรติชัยอะไหล่ยนต์ จำกัด ({branch_text}) (ผู้ซื้อ) ดังรายการต่อไปนี้", normal_style))
    story.append(Spacer(1, 10))

    # ---- table data
    # Your dataframe columns seen: DETAIL, QTY, UI, AMOUNT, MTP etc.
    # We'll interpret:
    #   - qty = QTY
    #   - unit = UI
    #   - unit price = AMOUNT / QTY  (if you prefer, use MTP or other)
    #   - line total = AMOUNT
    df = group_df.copy()

    # safe numeric
    df["QTY"] = pd.to_numeric(df.get("QTY", 0), errors="coerce").fillna(0)
    df["MTP"] = pd.to_numeric(df.get("MTP", 0), errors="coerce").fillna(0)
    df["AV_COST"] = pd.to_numeric(df.get("AV_COST", 0), errors="coerce").fillna(0)

    # UNIT_PRICE shown on the PDF = AV_COST
    df["UNIT_PRICE"] = df["AV_COST"]

    # AMOUNT used for the PDF (line total) = AV_COST * QTY * MTP
    df["AMOUNT_CALC"] = df["AV_COST"] * df["QTY"] * df["MTP"]


    headers = ["รหัสสินค้า", "รายการ", "ราคา/หน่วย", "จำนวน", "หน่วย", "รวมยอดเงิน"]
    table_rows = [headers]

    for _, r in df.iterrows():
        bcode = str(r.get("BCODE", ""))
        detail = str(r.get("DETAIL", ""))

        # Paragraph for wrapping long Thai descriptions
        detail_cell = Paragraph(detail.replace("\n", " "), small_style)

        unit_price = _money(r.get("UNIT_PRICE", ""))
        qty = _money(r.get("QTY", "")) if r.get("QTY", 0) % 1 else str(int(r.get("QTY", 0)))
        unit = str(r.get("UI", ""))
        total = _money(r.get("AMOUNT_CALC", ""))

        table_rows.append([bcode, detail_cell, unit_price, qty, unit, total])

    grand_total = float(df["AMOUNT_CALC"].sum())

    # Column widths tuned for A4
    col_widths = [70, 250, 70, 50, 45, 70]

    t = Table(table_rows, colWidths=col_widths, repeatRows=1)

    t.setStyle(TableStyle([
        ("FONTNAME", (0, 0), (-1, 0), font_bold),
        ("FONTSIZE", (0, 0), (-1, 0), 12),
        ("BACKGROUND", (0, 0), (-1, 0), colors.whitesmoke),
        ("ALIGN", (2, 1), (-1, -1), "RIGHT"),
        ("ALIGN", (0, 0), (-1, 0), "CENTER"),
        ("VALIGN", (0, 0), (-1, -1), "TOP"),
        ("GRID", (0, 0), (-1, -1), 0.5, colors.black),
        ("FONTNAME", (0, 1), (-1, -1), font_regular),
        ("FONTSIZE", (0, 1), (-1, -1), 11),
        ("LEFTPADDING", (0, 0), (-1, -1), 4),
        ("RIGHTPADDING", (0, 0), (-1, -1), 4),
        ("TOPPADDING", (0, 0), (-1, -1), 2),
        ("BOTTOMPADDING", (0, 0), (-1, -1), 2),
    ]))

    story.append(t)
    story.append(Spacer(1, 10))

    # ---- totals + signature area
    story.append(Paragraph(f"รวมทั้งสิ้น: {_money(grand_total)}", ParagraphStyle(
        "total_th", parent=styles["Normal"], fontName=font_bold, fontSize=13, leading=15, alignment=TA_RIGHT,
    )))
    story.append(Paragraph(f"จำนวนเงิน (ตัวอักษร): {thai_baht_text(grand_total)}", right_style))

    story.append(Spacer(1, 24))
    story.append(Paragraph("ผู้รับเงิน: _____________________", right_style))
    story.append(Spacer(1, 10))
    story.append(Paragraph("ผู้จ่ายเงิน: ____________________", right_style))
    story.append(Spacer(1, 10))
    story.append(Paragraph("หมายเหตุ: แนบสำเนาบัตรประชาชนผู้รับเงิน", right_style))
    doc.build(story)


def build_receipts_by_new_billno(df: pd.DataFrame, out_dir: str, *, font_dir: str | None = None):
    os.makedirs(out_dir, exist_ok=True)

    # Ensure NEW_BILLNO exists
    if "NEW_BILLNO" not in df.columns:
        raise ValueError("df must contain NEW_BILLNO column")

    for new_billno, g in df.groupby("NEW_BILLNO", sort=True):
        pdf_path = os.path.join(out_dir, f"{new_billno}.pdf")
        build_one_receipt_pdf(g, pdf_path, font_dir=font_dir)

    return out_dir


In [108]:
out_dir = build_receipts_by_new_billno(
    out_hq,
    out_dir="/content/drive/MyDrive/kcw_analytics/04_outputs/receipts_out",
    font_dir="/content/drive/MyDrive/kcw_analytics/00_fonts/Prompt"
)

print("Saved to:", out_dir)


Saved to: /content/drive/MyDrive/kcw_analytics/04_outputs/receipts_out
